In [1]:
import requests
import re
import time
import json
from pprint import pprint
import random
import pandas as pd
import datetime

Функция для получения данных

In [2]:
proxy = {
    'http': 'http://46.47.197.210:3128',
    'http': 'http://62.33.207.201:80'
}

def get_api_data(url, params = '', headers ='', proxy = ''): 
    r = requests.get(url, params, headers=headers, proxies=proxy)
    return r.json()

Функция для случайного выбора юзер-агента

In [3]:
def choose_user_agent(user_agents):
    user_agent = user_agents[random.randrange(len(user_agents))]
    headers = {'user-agent': user_agent}
    return headers

Чтение данных юзерагетов

In [4]:
with open(r'C:\Users\Lenovo\Desktop\Python ML\python\diplom\desktop_user_agent.txt', 'r', encoding='utf-8') as fp:
    user_agents = [line.strip() for line in fp.read().split('\n')]


Функция для получения данных о вакансиях за день

In [5]:
def get_vac_ids(date):
    url = r'https://api.hh.ru/vacancies'
    all_vac = []
    par = {'text': 'аналитик', 'area': 113, 'search_field': 'name', 'date_from': str(date), 'date_to': str(date)}
    #data = get_api_data(url, params=par, headers=choose_user_agent(user_agents))
    data = get_api_data(url, params=par, headers=choose_user_agent(user_agents))
    pages = data['pages']
    found = data['found']
    for page in range(pages):
        par = {'text': 'аналитик', 'area': 113, 'search_field': 'name', 'date_from': str(date), 'date_to': str(date), 'page': page}
        data = get_api_data(url, params=par, headers=choose_user_agent(user_agents))
        all_vac = all_vac + data['items']
    vac_ids = []
    for one_vac in all_vac:
        vac_ids.append(one_vac['id'])
    vac_ids.sort()
    return vac_ids

Проверка нет ли капчи

In [53]:
url = f'https://api.hh.ru/vacancies/74298375'
check = get_api_data(url, headers=choose_user_agent(user_agents))
#print(check)

Получение нужных данных о вакансиях

In [6]:
def get_vac_data(vac_ids):
    vac_info = []
    for one_vac in vac_ids:
        url = f'https://api.hh.ru/vacancies/{one_vac}'
        vac_descr = get_api_data(url, headers=choose_user_agent(user_agents))
        #print(vac_descr)
        try:
            area = vac_descr['area']['name']
            description = re.sub(r'\<[^>]*\>', '', vac_descr['description'])
            employment = vac_descr['employment']['name'],
            experience = vac_descr['experience']['name'],
            id = vac_descr['id'],
            key_skills = vac_descr['key_skills'],
            name = vac_descr['name'],
            publication_date = vac_descr['published_at'],
            salary = vac_descr['salary'],
            schedule = vac_descr['schedule']['name'],
            specializations = vac_descr['specializations'],
            proffesional_roles= vac_descr['professional_roles']
            employer = vac_descr['employer']['name']
        except:
            print(one_vac, 'ERRROR')
            continue
        vac_info.append({
            'area': area,
            'description': description,
            'employment': employment,
            'experience': experience,
            'id': id,
            'key_skills': key_skills,
            'name': name,
            'publication_date': publication_date,
            'salary': salary,
            'schedule': schedule,
            'specializations': specializations,
            'proffesional_roles': proffesional_roles,
            'employer': employer,
            'source': 'hh.ru'
        })
    return vac_info
    
    #time.sleep(0.1)


Функция для дописывания полученные данных за день в файл

In [7]:
def add_data_to_file(vac_info):
    with open(r'C:\Users\Lenovo\Desktop\Python ML\python\diplom\hh_data.json', 'r', encoding='utf-8') as fp:
        all_vacs = json.load(fp)
    all_vacs = all_vacs + vac_info
    with open(r'C:\Users\Lenovo\Desktop\Python ML\python\diplom\hh_data2.json', 'w', encoding='utf-8') as fp:
        json.dump(all_vacs, fp, ensure_ascii=False)

Функция, чтобы сохранить из json в excel

In [8]:
def from_json_to_excel():
    with open(r'C:\Users\Lenovo\Desktop\Python ML\python\diplom\hh_data.json', 'r', encoding='utf-8') as fp:
        to_excel = json.load(fp)
    df = pd.DataFrame(to_excel)
    df.to_excel(r'C:\Users\Lenovo\Desktop\Python ML\python\diplom\hh_data.xlsx')

Функция для добавления навыков в словарь и сохранения в файл

In [9]:
def add_skills_to_dict(vac_info, save):
    with open(r'C:\Users\Lenovo\Desktop\Python ML\python\diplom\skills_dict.json', 'r', encoding='utf-8') as fp:
        skills_dict = json.load(fp)
    for i, vac in enumerate(vac_info):
        if len(vac['key_skills'][0]) > 0:
            for skill in vac['key_skills'][0]:
                if skill['name'].lower() not in skills_dict:
                    skills_dict.append(skill['name'].lower())
    skills_dict.sort()
    if save == True:
        with open(r'C:\Users\Lenovo\Desktop\Python ML\python\diplom\skills_dict.json', 'w', encoding='utf-8') as fp:
            json.dump(skills_dict, fp, ensure_ascii=False)
    return skills_dict

In [31]:
ids = get_vac_ids(datetime.date.today() - datetime.timedelta(1))

In [33]:
vac_info = get_vac_data(ids)

In [38]:
add_data_to_file(vac_info)

In [78]:
from_json_to_excel()

In [39]:
with open(r'C:\Users\Lenovo\Desktop\Python ML\python\diplom\hh_data.json', 'r', encoding='utf-8') as fp:
        all_vacs = json.load(fp)

In [40]:
skills_dict = add_skills_to_dict(vac_info, True)

In [41]:
print(len(skills_dict))

4048
